In [6]:
import json
import os
import pandas as pd
from tqdm import tqdm

def loop_slices_to_csv(path, num_slices=20, output_csv='unique_tracks.csv'):
    cnt = 0
    cnt1 = 0
    mpd_playlists = []
    unique_tracks = pd.DataFrame()
    filenames = os.listdir(path)
    
    for fname in tqdm(sorted(filenames, key=len)):
        if fname.endswith(".json"):
            cnt += 1
            fullpath = os.path.join(path, fname)
            with open(fullpath, 'r') as file:
                current_slice = json.load(file)
            
            # Create a list of all playlists
            for playlist in current_slice['playlists']:
                cnt1 += 1
                mpd_playlists.append(playlist)
                
                if cnt1 == 1000:
                    cnt1 = 0
                    temp = pd.DataFrame(mpd_playlists)
                    temp = temp.explode('tracks')
                    temp = pd.concat([temp.drop(['tracks'], axis=1), temp['tracks'].apply(pd.Series)], axis=1)
                    unique_tracks = pd.concat([unique_tracks, temp], axis=0)
                    mpd_playlists = []

        if cnt == num_slices:
            break
    
    # Export the DataFrame to CSV
    unique_tracks.to_csv(output_csv, index=False)
    print(f"CSV file '{output_csv}' has been created with all the extracted data.")

path = '../Data/'
loop_slices_to_csv(path)


100%|██████████| 8/8 [00:24<00:00,  3.05s/it]


CSV file 'unique_tracks.csv' has been created with all the extracted data.


In [7]:
import pandas as pd
df = pd.read_csv('../Data/unique_tracks.csv')

In [9]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

In [10]:
import numpy as np

# Replace NaN values with an empty string in the specified columns
df['track_uri'].fillna('', inplace=True)
df['artist_uri'].fillna('', inplace=True)
df['album_uri'].fillna('', inplace=True)

# Convert columns to string type
df['track_uri'] = df['track_uri'].astype(str)
df['artist_uri'] = df['artist_uri'].astype(str)
df['album_uri'] = df['album_uri'].astype(str)

# Apply regex operations
df["track_uri"] = df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0] if x else np.nan)
df["artist_uri"] = df["artist_uri"].apply(lambda x: re.findall(r'\w+$', x)[0] if x else np.nan)
df["album_uri"] = df["album_uri"].apply(lambda x: re.findall(r'\w+$', x)[0] if x else np.nan)


In [14]:
# Spotify API credentials
client_id = 'd4eec2244fb2416ebce8ec488b1f9587'
client_secret = '0cf3c26eca6244e796a10a847dc11f17'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.client.Spotify(auth_manager=auth_manager)

In [12]:
t_uri=df["track_uri"].unique()
a_uri=df["artist_uri"].unique()

In [19]:
import json
import os
import pandas as pd
import datetime
import time
from tqdm import tqdm

f = open('../Data/audio_features.csv', 'a')
e = 0
for i in tqdm(range(0, len(t_uri), 100)):
    try:
        track_feature = sp.audio_features(t_uri[i:i + 100])
        track_df = pd.DataFrame(track_feature)
        # Add header only if the file is empty
        csv_data = track_df.to_csv(header=f.tell() == 0, index=False)
        f.write(csv_data)
    except Exception as error:
        e += 1
        with open("audio_features_log.txt", "a") as r:
            r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S") + ": " + str(error) + '\n')
        time.sleep(3)
        continue

with open("audio_features_log.txt", "a") as r:
    r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S") + " _________________________ " +
            "Total Number Of Errors : " + str(e) + " _________________________ " + '\n')

f.close()



100%|██████████| 663/663 [22:16<00:00,  2.02s/it]


In [21]:
df_aud = pd.read_csv('../Data/audio_features.csv')
df_aud

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.634,0.862,2,-5.337,1,0.0376,0.028700,0.000000,0.0217,0.767,95.989,audio_features,1zWZvrk13cL8Sl3VLeG57F,spotify:track:1zWZvrk13cL8Sl3VLeG57F,https://api.spotify.com/v1/tracks/1zWZvrk13cL8...,https://api.spotify.com/v1/audio-analysis/1zWZ...,227693,4
1,0.595,0.389,2,-9.245,1,0.0349,0.418000,0.000001,0.1160,0.400,166.052,audio_features,5kNe7PE09d6Kvw5pAsx23n,spotify:track:5kNe7PE09d6Kvw5pAsx23n,https://api.spotify.com/v1/tracks/5kNe7PE09d6K...,https://api.spotify.com/v1/audio-analysis/5kNe...,227427,4
2,0.543,0.837,11,-6.448,1,0.0608,0.117000,0.000000,0.0872,0.506,168.056,audio_features,6GsAD8PgHxmEuIPTG8GP3M,spotify:track:6GsAD8PgHxmEuIPTG8GP3M,https://api.spotify.com/v1/tracks/6GsAD8PgHxmE...,https://api.spotify.com/v1/audio-analysis/6GsA...,186080,4
3,0.688,0.702,7,-4.792,0,0.0499,0.021500,0.000000,0.1280,0.740,94.006,audio_features,6Knv6wdA0luoMUuuoYi2i1,spotify:track:6Knv6wdA0luoMUuuoYi2i1,https://api.spotify.com/v1/tracks/6Knv6wdA0luo...,https://api.spotify.com/v1/audio-analysis/6Knv...,192191,4
4,0.457,0.948,10,-3.364,1,0.0354,0.019100,0.000000,0.0536,0.878,148.000,audio_features,0i5el041vd6nxrGEU8QRxy,spotify:track:0i5el041vd6nxrGEU8QRxy,https://api.spotify.com/v1/tracks/0i5el041vd6n...,https://api.spotify.com/v1/audio-analysis/0i5e...,208960,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51339,0.916,0.742,7,-12.440,1,0.1880,0.000288,0.004980,0.0986,0.545,127.809,audio_features,2PBTwMH2mzfLigdMyPzOcp,spotify:track:2PBTwMH2mzfLigdMyPzOcp,https://api.spotify.com/v1/tracks/2PBTwMH2mzfL...,https://api.spotify.com/v1/audio-analysis/2PBT...,257640,4
51340,0.667,0.916,6,-3.264,1,0.0743,0.165000,0.000011,0.1620,0.558,100.019,audio_features,5JugcqxQihVYdvCSPzmP1H,spotify:track:5JugcqxQihVYdvCSPzmP1H,https://api.spotify.com/v1/tracks/5JugcqxQihVY...,https://api.spotify.com/v1/audio-analysis/5Jug...,183559,4
51341,0.775,0.602,8,-10.732,0,0.1390,0.000280,0.878000,0.1740,0.552,136.965,audio_features,1cKRBp7hrBVD4eP3W9x2AI,spotify:track:1cKRBp7hrBVD4eP3W9x2AI,https://api.spotify.com/v1/tracks/1cKRBp7hrBVD...,https://api.spotify.com/v1/audio-analysis/1cKR...,412427,4
51342,0.741,0.841,2,-6.398,0,0.2160,0.070200,0.000000,0.1460,0.730,94.846,audio_features,38griAVM808crjbFp9gcPD,spotify:track:38griAVM808crjbFp9gcPD,https://api.spotify.com/v1/tracks/38griAVM808c...,https://api.spotify.com/v1/audio-analysis/38gr...,339573,4


In [28]:
# Open file in write mode for the first iteration to write headers
with open('../Data/track_features.csv', 'w') as f:
    for i in tqdm(range(0, len(t_uri), 50)):
        try:
            track_features = sp.tracks(t_uri[i:i + 50])
            for x in range(50):
                track_pop = pd.DataFrame([t_uri[i + x]], columns=['track_uri'])  # Assigning column name 'track_uri'
                track_pop['release_date'] = track_features['tracks'][x]['album']['release_date']
                track_pop['popularity'] = track_features['tracks'][x]["popularity"]  # Renaming 'pop' to 'popularity'
                
                csv_data = track_pop.to_csv(header=f.tell() == 0, index=False)  # Write header only if it's the first write
                f.write(csv_data)
            time.sleep(0.5)  # Introduce a delay between requests (adjust as needed)
        except Exception as error:
            # Error handling
            continue

# Read the CSV into a DataFrame
df_trc = pd.read_csv('../Data/track_features.csv')
df_trc


100%|██████████| 1325/1325 [21:20<00:00,  1.03it/s]


,track_uri,release_date,popularity
0,31kTKEPu5PPTTODydIYai0,2016-04-29,20
1,54i6YAQkZImwWjmzPrQeqd,2016-06-24,0
2,1UqhkbzB1kuFwt2iy4h29Q,2011,72
3,03f7xZmt2wHCIDJBFPK8G4,2015-08-14,74
4,3ASt4jBSatnN4RGMlym74h,2014-10-27,13
...,...,...,...
65932,2PBTwMH2mzfLigdMyPzOcp,1991-09-03,44
65933,5JugcqxQihVYdvCSPzmP1H,2016-09-09,0
65934,1cKRBp7hrBVD4eP3W9x2AI,1998,0
65935,38griAVM808crjbFp9gcPD,2003,15


In [30]:
# Open file in write mode for the first iteration to write headers
with open('../Data/artist_features.csv', 'w') as f:
    e = 0
    for i in tqdm(range(0, len(a_uri), 50)):
        try:
            artist_features = sp.artists(a_uri[i:i + 50])
            for x in range(50):
                artist_df = pd.DataFrame([a_uri[i + x]])
                artist_pop = artist_features['artists'][x]["popularity"]
                artist_genres = artist_features['artists'][x]["genres"]
                artist_df["artist_pop"] = artist_pop
                if artist_genres: 
                    artist_df["genres"] = " ".join([re.sub(' ', '_', i) for i in artist_genres])
                else:
                    artist_df["genres"] = "unknown"
                csv_data = artist_df.to_csv(header=f.tell() == 0, index=False)  # Write header only if it's the first write
                f.write(csv_data)
            time.sleep(0.5)  # Introduce a delay between requests (adjust as needed)
        except Exception as error:
            e += 1
            with open("artist_features.txt", "a") as r:
                r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S") + ": " + str(error) + '\n')
            time.sleep(3)
            continue

    with open("artist_features.txt", "a") as r:
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S") + " _________________________ " +
                "Total Number Of Errors : " + str(e) + " _________________________ " + '\n')


100%|██████████| 282/282 [04:11<00:00,  1.12it/s]


,track_uri,release_date,popularity
0,31kTKEPu5PPTTODydIYai0,2016-04-29,20
1,54i6YAQkZImwWjmzPrQeqd,2016-06-24,0
2,1UqhkbzB1kuFwt2iy4h29Q,2011,72
3,03f7xZmt2wHCIDJBFPK8G4,2015-08-14,74
4,3ASt4jBSatnN4RGMlym74h,2014-10-27,13
...,...,...,...
65932,2PBTwMH2mzfLigdMyPzOcp,1991-09-03,44
65933,5JugcqxQihVYdvCSPzmP1H,2016-09-09,0
65934,1cKRBp7hrBVD4eP3W9x2AI,1998,0
65935,38griAVM808crjbFp9gcPD,2003,15


In [31]:
df_art = pd.read_csv('../Data/artist_features.csv')
df_art


,0,artist_pop,genres
0,4tX2TplrkIP4v05BNC903e,67,album_rock classic_rock hard_rock heartland_ro...
1,4NGiEU3Pkd8ASRyQR30jcA,50,acoustic_pop neo_mellow pop_rock
2,5cTn7jRVGqTdmgMW75FIal,30,indie_poptimism shimmer_pop
3,1g1yxsNVPhMUl9GrMjEb2o,61,modern_rock neo_mellow neon_pop_punk pop_punk ...
4,5cIc3SBFuBLVxJz58W2tU9,65,electropop indie_poptimism pov:_indie
...,...,...,...
14044,0OirGRY55NlQeqOHKhvpbm,27,bass_trap traprun
14045,6Ju8rLVGq0MMt0NaXyfLJJ,18,vaporwave
14046,1YMADOVqFTlVjZwbCOSp6c,2,unknown
14047,0aNCjE72yyrhKQB1qfPBpi,58,eurodance
